In [154]:
import pandas as pd
from sklearn.model_selection import train_test_split
import math

## Importamos los datos y los ajustamos para tener menos variables y observaciones

In [14]:
df = pd.read_csv('wine_quality.csv',sep=";")
df.columns = [x.replace(' ','_') for x in df.columns]
df = df[['residual_sugar','pH','fixed_acidity','quality']]
data = df.sample(n=200).reset_index(drop=True)
y = data.loc[:,'quality']
X = data.drop(['quality'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [15]:
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

### Clases prestadas de CART
##### Las siguientes clases fueron adaptadas para ser usadas con un data_frame y para un árbol de decisión de regresión

In [51]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            self.column, condition, str(self.value))

In [89]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [106]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows,y_train):
        self.average = self.calculate_mean(rows,y_train)
        
    def calculate_mean(self,rows,y_train):
        sum_acc = 0
        n = len(rows)
        for i in rows:
            sum_acc += y_train[i]
            
        return sum_acc/n

#### Clase Decision_Tree
##### Se tomaron los componentes de CART como partition, find_best_split y build_tree pero fueron adaptadas para un árbol de decisión de regresión además de compactadas en una sola clase para mejor implementación. En lugar del coeficiente gini, se utiliza el Sum of Squared Errors para determinar qué partición es la mejor (Menor SSE) por lo cual se encuentra también implementada SSE.

In [160]:
class Decision_Tree:
    def __init__(self,X_train,y_train,min_split):
        self.x_train = X_train
        self.y_train = y_train
        self.root = None
        self.min_split = min_split
        self.features = X_train.columns
        
    
    def partition(self,question,rows):
        """Partitions a dataset.

        For each row in the dataset, check if it matches the question. If
        so, add it to 'true rows', otherwise, add it to 'false rows'.
        """
        true_rows, false_rows = [], []
     
        for row in rows:
            observation = self.x_train.loc[row]
            #print(observation)
            if question.match(observation):
                true_rows.append(row)
            else:
                false_rows.append(row)
   
        return true_rows, false_rows
    
    def find_best_split(self,rows):
        """Find the best question to ask by iterating over every feature / value
        and calculating the information gain."""
        least_loss = 100000000000000  # keep track of the best information gain
        best_question = None  # keep train of the feature / value that produced it
        #n_features = len(rows[0]) - 1  # number of columns

        for col in self.features:  # for each feature

            values = set([self.x_train.loc[row][col] for row in rows])  # unique values in the column

            for val in values:  # for each value

                question = Question(col, val)
                
                # try splitting the dataset
                true_rows, false_rows = self.partition(question,rows)

                # Skip this split if it doesn't divide the
                # dataset.
                if len(true_rows) == 0 or len(false_rows) == 0:
                    continue

                # Calculate the information gain from this split
                loss = self.SSE(true_rows, false_rows)
                #print(loss)
                # You actually can use '>' instead of '>=' here
                # but I wanted the tree to look a certain way for our
                # toy dataset.
                if loss <= least_loss:
                    least_loss, best_question = loss, question

        return least_loss, best_question
    
    def SSE(self,true_rows,false_rows): #recibe numero de row
        mean_right = self.row_average(true_rows)
        #print(mean_right)
        mean_left = self.row_average(false_rows)
        #print(mean_left)
        ssr = self.sum_squared(true_rows,mean_right)
        ssl = self.sum_squared(false_rows,mean_left)
        return ssr+ssl

    def sum_squared(self,rows,mean):
        sum_acc = 0
        for i in rows:
            real = self.y_train[i]
            sum_acc += (real-mean)**2
        return sum_acc

    def row_average(self,rows):
        acc_sum = 0
        n = len(rows)
        for i in rows:
            acc_sum += self.y_train[i]
        return acc_sum/n
    
    def train(self):
        rows = list(range(0,self.x_train.shape[0]))
        self.root = self.build_tree(rows)
        
        
        
    def build_tree(self,rows):
        loss, question = self.find_best_split(rows)
        #print(question)
        # Base case: no further info gain
        # Since we can ask no further questions,
        # we'll return a leaf.
        #print(question)
        if len(rows)<self.min_split: #si hay muy pocas rows de este lado para dividir ya no lo divido y hago mayoría de votos
            return Leaf(rows,self.y_train)

        # If we reach here, we have found a useful feature / value
        # to partition on.
        true_rows, false_rows = self.partition(question,rows)

        # Recursively build the true branch.
        true_branch = self.build_tree(true_rows)

        # Recursively build the false branch.
        false_branch = self.build_tree(false_rows)

        # Return a Question node.
        # This records the best feature / value to ask at this point,
        # as well as the branches to follow
        # dependingo on the answer.
        return Decision_Node(question, true_branch, false_branch)
    
    def print_tree(self):
        self.__print_tree(self.root)
    
    def __print_tree(self,node, spacing=""):
        """World's most elegant tree printing function."""

        # Base case: we've reached a leaf
        if isinstance(node, Leaf):
            print (spacing + "Predict", node.average)
            return

        # Print the question at this node
        print (spacing + str(node.question))

        # Call this function recursively on the true branch
        print (spacing + '--> True:')
        self.__print_tree(node.true_branch, spacing + "  ")

        # Call this function recursively on the false branch
        print (spacing + '--> False:')
        self.__print_tree(node.false_branch, spacing + "  ")
    
    def predict(self,row, node):
        """See the 'rules of recursion' above."""

        # Base case: we've reached a leaf
        if isinstance(node, Leaf):
            return node.average

        # Decide whether to follow the true-branch or the false-branch.
        # Compare the feature / value stored in the node,
        # to the example we're considering.
        if node.question.match(row):
            return self.predict(row, node.true_branch)
        else:
            return self.predict(row, node.false_branch)
        
    def test_tree(self,x_test,y_test):
        y_pred = []
        n = x_test.shape[0]
        node = dt.root
        sum_acc_mae = 0
        sum_acc_rmse = 0
        for i in range(0,n):
            x = x_test.iloc[i]
            predicted_value = self.predict(x,node)
            y_pred.append(predicted_value)
            y_real = y_test[i]
            diff = (y_real-predicted_value)
            sum_acc_mae += abs(diff)
            sum_acc_rmse += diff**2
            
        RMSE = math.sqrt(sum_acc_rmse/n)  
        MAE = sum_acc_mae/n
        return y_pred,MAE,RMSE
        
        
        
    

### Implementación

In [162]:
dt = Decision_Tree(X_train,y_train,10) #creamos un nuevo árbol con X_Train, y_train y min_split = 10. min_split se puede variar para compensar bias y variance
# un menor min_split implica más variance mientras un mayor min_split más bias
dt.train()

#### Probamos el árbol con el set de prueba para ver su MAE y su RMSE

In [163]:
y_pred,MAE,RMSE = dt.test_tree(X_test,y_test)

In [164]:
MAE

0.8296115921115917

In [165]:
RMSE

1.0059723785620711

##### Podemos notar que son muy buenas ambas métricas ya que son muy pequeñas.